In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
import pandexo.engine.justdoit as jdi 
import os


# Modern Earth

In [ ]:
import astropy.units as u
earth_radius = u.R_earth.to(u.km)
sun_radius = u.R_sun.to(u.km)

## trappist 1 e spectrum to (rp/rs)^2
trappist1e_radius = 0.92 * earth_radius
trappist1_sun_radius = 0.1192 * sun_radius

spectrum = pd.read_csv('Earth_through_geologic_time_0.0Ga.csv') ## Taken from Kaltenegger et al. 2020
print(spectrum)
spectrum.columns = ['wavelength', 'effective_height']
trappist1e_rp_rs2 = (spectrum['effective_height'] + trappist1e_radius) ** 2 / trappist1_sun_radius ** 2
spectrum['(rp/rs)^2'] = trappist1e_rp_rs2

## reduct the number of points to 10_000
# Número de puntos a muestrear
n_points = 10000

# Generar índices equidistantes para el muestreo
indices = np.linspace(0, len(spectrum) - 1, n_points).astype(int)

# Muestrear el DataFrame
sampled_spectrum = spectrum.iloc[indices]


## plot the spectrum
plt.figure(figsize=(15,5))
plt.plot(sampled_spectrum['wavelength'], sampled_spectrum['(rp/rs)^2']*1e3, label=f'Trappist 1e spectrum 0.0 Ga Ago')
plt.xlabel('wavelength (micron)')
plt.ylabel('$(rp/rs)^2$  1e-3')
plt.title('Trappist 1e spectrum')
plt.show()

## save spectrum to a file txt, to be used in pandexo, only wavelength and (rp/rs)^2
sampled_spectrum[['wavelength', '(rp/rs)^2']].to_csv(
    f'E_0.0Ga.txt',
    index=False, sep=' ', header=False)
        
    

In [ ]:
n_transits= [1,2,3,4,5,6,7,8,9,10,20,50,100
             ]
obs = [ [] for _ in range(len(n_transits))] 

exo_dict = jdi.load_exo_dict()
exo_dict['observation']['sat_level'] = 80    #saturation level in percent of full well
exo_dict['observation']['sat_unit'] = '%'
exo_dict['observation']['noccultations'] = n_transits #number of transits
exo_dict['observation']['R'] = None          #fixed binning. I usually suggest ZERO binning.. you can always bin later
                                            #without having to redo the calcualtion
exo_dict['observation']['baseline_unit'] = 'total'  #Defines how you specify out of transit observing time
                                                    #'frac' : fraction of time in transit versus out = in/out
                                                    #'total' : total observing time (seconds)
exo_dict['observation']['baseline'] = 0.9535*3*60.0*60.0 #in accordance with what was specified above (total observing time)

exo_dict['observation']['noise_floor'] = 0   #this can be a fixed level or it can be a filepath
                                            #to a wavelength dependent noise floor solution (units are ppm)
                                            
exo_dict['star']['type'] = 'phoenix'        #phoenix or user (if you have your own)
exo_dict['star']['mag'] = 11.354               #magnitude of the system
exo_dict['star']['ref_wave'] = 1.25         #For J mag = 1.25, H = 1.6, K =2.22.. etc (all in micron)
exo_dict['star']['temp'] = 2566             #in K
exo_dict['star']['metal'] = 0.0             # as log Fe/H
exo_dict['star']['logg'] = 5.2396              #log surface gravity cgs

exo_dict['planet']['w_unit'] = 'um'                      #other options include "um","nm" ,"Angs", "sec" (for phase curves)
exo_dict['planet']['f_unit'] = 'rp^2/r*^2'               #other options are 'fp/f*'
exo_dict['planet']['transit_duration'] = 0.9535*60.0*60.0   #transit duration
exo_dict['planet']['td_unit'] = 's'                      #Any unit of time in accordance with astropy.units can be added
inst_dict=jdi.load_mode_dict('NIRSpec Prism')
inst_dict["configuration"]["detector"]["subarray"]="sub512"
inst_dict["configuration"]["detector"]["ngroup"] =6

exo_dict['planet']['exopath'] = f'E_0.0Ga.txt'

for i,n in enumerate(n_transits):
    print(f'Running {n} transits')
    print (f"i = {i}, n = {n}")
    exo_dict['observation']['noccultations'] = n
    results=jdi.run_pandexo(exo_dict, inst_dict)
    obs_copy = []

    for j in range(1000):
        obs_copy.append(np.random.normal(results["FinalSpectrum"]["spectrum"], results["FinalSpectrum"]["error_w_floor"]))

    waves = results["FinalSpectrum"]["wave"][18:]
    
    df = pd.DataFrame(obs_copy)
    df = df.iloc[:,18:]
    df.columns = waves
    ## save to a file
    df.to_csv(f'E_0.0Ga_{n}_transits.csv', index=False)
    
spec= pd.read_csv(f'E_0.0Ga.txt', sep=' ', header=None)
plt.figure(figsize=(15,5))
plt.plot(spec[0], spec[1]*1e3, label=f'Trappist 1e spectrum 0.0 Ga Ago')
plt.xlabel('wavelength (micron)')
plt.ylabel('$(rp/rs)^2$  1e-3')
plt.title('Trappist 1e spectrum')
## plot the results
plt.scatter(df.columns.values, df.iloc[0]*1e3, label=f'{n_transits[0]} transits',alpha=0.7,
            color='red', s=1)
plt.legend()
plt.xlim(min(df.columns.values), max(df.columns.values))
plt.show()

In [ ]:
import ast
def string_to_list(string):
    return ast.literal_eval(string)

## plot the real spectrum
plt.figure(figsize=(20,7))
spec = pd.read_csv(f'E_0.0Ga.txt', sep=' ', header=None)
plt.plot(spec[0], spec[1]*1e3, label=f'Trappist 1e spectrum 0.0 Ga',
            alpha=0.5, color='blue')
## plot the results 100 transits
df = pd.read_csv(f'E_0.0Ga_100_transits.csv')
cols = df.columns.values
cols = [float(i) for i in cols]

for i in range(100):
    plt.scatter(cols, df.iloc[i]*1e3, alpha=0.1, color='red', s=1)


plt.xlim(min(cols), max(cols))
plt.legend()
plt.legend()
plt.title(f'Trappist 1e spectrum: 0.0 Ga Ago')
plt.show()

# Proterozoic

In [ ]:
import astropy.units as u
earth_radius = u.R_earth.to(u.km)
sun_radius = u.R_sun.to(u.km)

## trappist 1 e spectrum to (rp/rs)^2
trappist1e_radius = 0.92 * earth_radius
trappist1_sun_radius = 0.1192 * sun_radius

spectrum = pd.read_csv('Earth_through_geologic_time_2.0Ga.csv') ## Taken from Kaltenegger et al. 2020
print(spectrum)
spectrum.columns = ['wavelength', 'effective_height']
trappist1e_rp_rs2 = (spectrum['effective_height'] + trappist1e_radius) ** 2 / trappist1_sun_radius ** 2
spectrum['(rp/rs)^2'] = trappist1e_rp_rs2

## reduct the number of points to 10_000
# Número de puntos a muestrear
n_points = 10000

# Generar índices equidistantes para el muestreo
indices = np.linspace(0, len(spectrum) - 1, n_points).astype(int)

# Muestrear el DataFrame
sampled_spectrum = spectrum.iloc[indices]


## plot the spectrum
plt.figure(figsize=(15,5))
plt.plot(sampled_spectrum['wavelength'], sampled_spectrum['(rp/rs)^2']*1e3, label=f'Trappist 1e spectrum 0.0 Ga Ago')
plt.xlabel('wavelength (micron)')
plt.ylabel('$(rp/rs)^2$  1e-3')
plt.title('Trappist 1e spectrum')
plt.show()

## save spectrum to a file txt, to be used in pandexo, only wavelength and (rp/rs)^2
sampled_spectrum[['wavelength', '(rp/rs)^2']].to_csv(
    f'E_2.0Ga.txt',
    index=False, sep=' ', header=False)
        
    

In [ ]:
n_transits= [1,2,3,4,5,6,7,8,9,10,20,50,100
             ]
obs = [ [] for _ in range(len(n_transits))] 

exo_dict = jdi.load_exo_dict()
exo_dict['observation']['sat_level'] = 80    #saturation level in percent of full well
exo_dict['observation']['sat_unit'] = '%'
exo_dict['observation']['noccultations'] = n_transits #number of transits
exo_dict['observation']['R'] = None          #fixed binning. I usually suggest ZERO binning.. you can always bin later
                                            #without having to redo the calcualtion
exo_dict['observation']['baseline_unit'] = 'total'  #Defines how you specify out of transit observing time
                                                    #'frac' : fraction of time in transit versus out = in/out
                                                    #'total' : total observing time (seconds)
exo_dict['observation']['baseline'] = 0.9535*3*60.0*60.0 #in accordance with what was specified above (total observing time)

exo_dict['observation']['noise_floor'] = 0   #this can be a fixed level or it can be a filepath
                                            #to a wavelength dependent noise floor solution (units are ppm)
                                            
exo_dict['star']['type'] = 'phoenix'        #phoenix or user (if you have your own)
exo_dict['star']['mag'] = 11.354               #magnitude of the system
exo_dict['star']['ref_wave'] = 1.25         #For J mag = 1.25, H = 1.6, K =2.22.. etc (all in micron)
exo_dict['star']['temp'] = 2566             #in K
exo_dict['star']['metal'] = 0.0             # as log Fe/H
exo_dict['star']['logg'] = 5.2396              #log surface gravity cgs

exo_dict['planet']['w_unit'] = 'um'                      #other options include "um","nm" ,"Angs", "sec" (for phase curves)
exo_dict['planet']['f_unit'] = 'rp^2/r*^2'               #other options are 'fp/f*'
exo_dict['planet']['transit_duration'] = 0.9535*60.0*60.0   #transit duration
exo_dict['planet']['td_unit'] = 's'                      #Any unit of time in accordance with astropy.units can be added
inst_dict=jdi.load_mode_dict('NIRSpec Prism')
inst_dict["configuration"]["detector"]["subarray"]="sub512"
inst_dict["configuration"]["detector"]["ngroup"] =6

exo_dict['planet']['exopath'] = f'E_2.0Ga.txt'

for i,n in enumerate(n_transits):
    print(f'Running {n} transits')
    print (f"i = {i}, n = {n}")
    exo_dict['observation']['noccultations'] = n
    results=jdi.run_pandexo(exo_dict, inst_dict)
    obs_copy = []

    for j in range(1000):
        obs_copy.append(np.random.normal(results["FinalSpectrum"]["spectrum"], results["FinalSpectrum"]["error_w_floor"]))

    waves = results["FinalSpectrum"]["wave"][18:]
    
    df = pd.DataFrame(obs_copy)
    df = df.iloc[:,18:]
    df.columns = waves
    ## save to a file
    df.to_csv(f'E_0.0Ga_{n}_transits.csv', index=False)
    
spec= pd.read_csv(f'E_0.0Ga.txt', sep=' ', header=None)
plt.figure(figsize=(15,5))
plt.plot(spec[0], spec[1]*1e3, label=f'Trappist 1e spectrum 0.0 Ga Ago')
plt.xlabel('wavelength (micron)')
plt.ylabel('$(rp/rs)^2$  1e-3')
plt.title('Trappist 1e spectrum')
## plot the results
plt.scatter(df.columns.values, df.iloc[0]*1e3, label=f'{n_transits[0]} transits',alpha=0.7,
            color='red', s=1)
plt.legend()
plt.xlim(min(df.columns.values), max(df.columns.values))
plt.show()

In [ ]:
import ast
def string_to_list(string):
    return ast.literal_eval(string)

## plot the real spectrum
plt.figure(figsize=(20,7))
spec = pd.read_csv(f'E_2.0Ga.txt', sep=' ', header=None)
plt.plot(spec[0], spec[1]*1e3, label=f'Trappist 1e spectrum 0.0 Ga',
            alpha=0.5, color='blue')
## plot the results 100 transits
df = pd.read_csv(f'E_2.0Ga_100_transits.csv')
cols = df.columns.values
cols = [float(i) for i in cols]

for i in range(100):
    plt.scatter(cols, df.iloc[i]*1e3, alpha=0.1, color='red', s=1)


plt.xlim(min(cols), max(cols))
plt.legend()
plt.legend()
plt.title(f'Trappist 1e spectrum: 2.0 Ga Ago')
plt.show()